<a href="https://colab.research.google.com/github/khatriprajwol/Data-Visualization-/blob/main/Quiz%205%20Sub-Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quiz 5. Sub-Queries.

In this project, I will be using the same data which I have used in the previous project. The data is about the 'austin_bikeshare.bikeshare_trips'.

In [89]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


This code will is used to authorize the permission to use the data which included several process. 

In [90]:
%%bigquery --project data-base-1-338717
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,total_rows
0,1424786


There is total of 1424786 rows in the data set which is a lot. Lets have a look at the data and find out the attributes.

In [107]:
%%bigquery --project data-base-1-338717
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 5

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


This input gives me only 5 rows even though it has total of 1424786 rows. That is really helpful to over view in a small table which beautifully presents the information. 

In [110]:
%%bigquery --project data-base-1-338717

SELECT COUNT(*) as start_station_id, start_station_name
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY start_station_id DESC

,start_station_id,start_station_name
0,72799,21st & Speedway @PCL
1,40635,Riverside @ S. Lamar
2,36520,City Hall / Lavaca & 2nd
3,35307,2nd & Congress
4,34758,Rainey St @ Cummings
...,...,...
188,4,Marketing Event
189,2,Eeyore's 2018
190,1,Stolen
191,1,Eeyore's 2017


In [119]:
%%bigquery --project data-base-1-338717

SELECT MAX(start_station_id) as max_start_station_id
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = "21st & Speedway @PCL"

,max_start_station_id
0,3798


In [121]:
%%bigquery --project data-base-1-338717
SELECT MAX(start_station_id) as second_most_popular_starting_station
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_id != (SELECT MAX(start_station_id) as max_start_station_id FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips` WHERE start_station_name = "21st & Speedway @PCL")
      AND start_station_name = "Riverside @ S. Lamar"

,second_most_popular_starting_station
0,2575


The *second_most_popular_starting_station* is 2575. 

In [122]:
%%bigquery --project data-base-1-338717

SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
LIMIT 100

,start_station_name,duration_minutes
0,Toomey Rd @ South Lamar,31
1,Toomey Rd @ South Lamar,31
2,Toomey Rd @ South Lamar,30
3,State Capitol @ 14th & Colorado,19
4,State Capitol @ 14th & Colorado,17
...,...,...
95,Rainey @ River St,87
96,Zilker Park West,11
97,ACC - West & 12th Street,17
98,Rainey @ River St,87


In [126]:
%%bigquery --project data-base-1-338717

SELECT start_station_name, COUNTIF (duration_minutes > 60) as trip_lasted_over_an_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name , duration_minutes
Order BY trip_lasted_over_an_hour DESC

,start_station_name,trip_lasted_over_an_hour
0,Riverside @ S. Lamar,185
1,Riverside @ S. Lamar,173
2,Zilker Park,169
3,Riverside @ S. Lamar,150
4,Rainey St @ Cummings,145
...,...,...
40930,Nueces & 26th,0
40931,Nueces & 26th,0
40932,26th/Nueces,0
40933,Nueces & 26th,0


We can clearly see that River @S. Lamar lasted for a long time. I think it is hard to cycle for that period of time. 

I will make a tabe asking how many trips lasted more than an hour. 

In [135]:
%%bigquery --project data-base-1-338717

SELECT *, (CASE WHEN  duration_minutes>60 THEN 1 ELSE 0 END) as over_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 100

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes,over_hour
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39,0
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31,0
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31,0
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30,0
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19,0
...,...,...,...,...,...,...,...,...,...,...
95,9900290499,Local365,100,2015-10-02 17:12:03+00:00,1008,Nueces @ 3rd,1006,Zilker Park West,11,0
96,9900290570,Weekender,58,2015-10-03 13:12:11+00:00,2576,Rainey @ River St,1006,Zilker Park West,23,0
97,9900290601,Walk Up,963,2015-10-03 21:12:51+00:00,1006,Zilker Park West,2546,ACC - West & 12th Street,18,0
98,9900290615,Walk Up,509,2015-10-03 13:12:22+00:00,1007,Lavaca & 6th,3291,11th & San Jacinto,17,0


In [138]:
%%bigquery --project data-base-1-338717

SELECT start_station_name , COUNTIF(duration_minutes>60) as trip_lasted_over_an_hour , COUNT(*) as number_of_trips_over_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name, subscriber_type
Order BY trip_lasted_over_an_hour DESC

,start_station_name,trip_lasted_over_an_hour,number_of_trips_over_hour
0,Riverside @ S. Lamar,3888,22093
1,Rainey St @ Cummings,3485,17237
2,2nd & Congress,2954,16057
3,Zilker Park,2613,15581
4,Davis at Rainey Street,2270,15408
...,...,...,...
4480,26th/Nueces,0,5
4481,Nueces & 26th,0,2
4482,Nueces & 26th,0,44
4483,Nueces & 26th,0,1


I am not sure if the last two columns data is supposed to be same. I hope it makes sense. 